# 사전학습 모델 테스트 (유의미 VS 무의미 classification)

- insightful keyword
- 긍/ 부정
- 장르 분류
- 검색 매칭


## 사전학습 모델 : 검색 매칭 모델
- 쿼리에 대한 문단의 관련도
- amberoad/bert-multilingual-passage-reranking-msmarco
- https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco?text=i+love+you

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
train = pd.read_csv(f"{DATA_PATH}뉴진스_202306_200p.csv")

In [ ]:
train = pd.DataFrame(train)

In [ ]:
text = train["title"][0]

In [ ]:
# train = tokenizer(train['title'].tolist(),truncation=True,padding="max_length")

In [ ]:
news_title = []

for sentence in train['title']:
    news_title.append(sentence)


In [ ]:
news_title

['\'새로운 유전자\' 뉴진스 "K팝, 언어장벽 넘어 소통하는 매개체"',
 '뉴진스도 아니고 피프티피프티 정산주장, 사실상 배은망덕 낙인',
 '뉴진스, 비치 백에 CD 담은 파격"소장 욕구 자극"',
 "Y현장 'R U Next?', 르세라핌뉴진스 만든 하이브 걸그룹 선발법 다를까",
 "뉴진스 후배 뽑는 과정 대공개하이브 오디션 '알유넥스트'종합",
 '팝\'s현장"뉴진스 사랑해"..하이브 시스템 업은 \'알유넥스트\', 보석 발굴할까',
 '소녀시대카라2AM이 찾는 르세라핌뉴진스 후배 걸그룹 어떨까 종합',
 "BTS르세라핌뉴진스의 후배들'알유넥스트', 소녀들의 여정 시작 종합",
 '방시혁 "르세라핌뉴진스 잇는다"\'알 유 넥스트\' 기대되는 이유',
 'R U Next? BTS뉴진스, 그 다음은 누구?종합',
 "르세라핌뉴진스 이을까..'알유넥스트', 소시카라도 응원한 하이브 걸그룹종합",
 "'알 유 넥스트?' 르세라핌뉴진스 이을 하이브  걸그룹 첫 발종합 SE현장",
 '르세라핌뉴진스 다음은?알유넥스트 서바이벌 표방 청춘 성장물종합',
 '포토뉴진스 민지, 옆모습도 예술',
 '미션 임파서블7 헤일리 앳웰, 서울 떠나며 뉴진스 음악 들어요 감사 인사',
 "뉴진스도 빠진 '발레코어' 뭐길래 리본메리제인 인기 UP",
 '조권 "르세라핌뉴진스 잇는 \'알유넥스트\', 욕심과 열정 보인다"',
 '뉴진스, 금발amp흑발 요정',
 '뉴진스엔믹스있지그룹 7월대전 발발 "\'서머퀸\' 정조준"초점S',
 '뉴진스르세라핌 이은 하이브 걸그룹 현미경 JTBC R U NEXT?종합',
 '"잘봐 이게 제2의 뉴진스다"\'R U Next?\' 오늘 첫 방',
 "아이키 '뉴진스 말고 누진스'엑's HD포토",
 '박준금, 뉴진스도 입을 법한 휴양지룩 뒤태만 보면 나이 몰라매거진준금',
 '60세 박준금, 완벽 뒤태 자랑.. MZ세대뉴진스 부럽지 않은 패션 센스',
 '다큐처럼R U Next? 최수영조권의 제2의 뉴진스 찾기 종합MK현장',
 '제2의 뉴진스 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd


In [ ]:
# 관련도 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")
model = AutoModelForSequenceClassification.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")

In [ ]:

# 결과 저장할 리스트 초기화
scores = []

# 모델에 각 기사 제목에 대한 관련도를 계산하여 scores에 저장
for title in news_title:
    # 쿼리와 문단 생성 (이 모델은 passage reranking을 위한 모델이므로 쿼리는 고정값으로 사용)
    query = "광고 모델"
    passage = title
    inputs = tokenizer.encode_plus(query, passage, return_tensors="pt", padding=True, truncation=True)

    # 모델 추론
    outputs = model(**inputs)
    score = outputs.logits[0][0].item()
    scores.append(score)

# 결과 출력
for i, title in enumerate(news_title):
    print(f"기사 제목: {title}, 관련도 점수: {scores[i]}")

기사 제목: '새로운 유전자' 뉴진스 "K팝, 언어장벽 넘어 소통하는 매개체", 관련도 점수: 4.305500030517578
기사 제목: 뉴진스도 아니고 피프티피프티 정산주장, 사실상 배은망덕 낙인, 관련도 점수: 3.6231849193573
기사 제목: 뉴진스, 비치 백에 CD 담은 파격"소장 욕구 자극", 관련도 점수: 4.340878009796143
기사 제목: Y현장 'R U Next?', 르세라핌뉴진스 만든 하이브 걸그룹 선발법 다를까, 관련도 점수: 4.4918718338012695
기사 제목: 뉴진스 후배 뽑는 과정 대공개하이브 오디션 '알유넥스트'종합, 관련도 점수: 4.127279758453369
기사 제목: 팝's현장"뉴진스 사랑해"..하이브 시스템 업은 '알유넥스트', 보석 발굴할까, 관련도 점수: 4.575690746307373
기사 제목: 소녀시대카라2AM이 찾는 르세라핌뉴진스 후배 걸그룹 어떨까 종합, 관련도 점수: 4.121578693389893
기사 제목: BTS르세라핌뉴진스의 후배들'알유넥스트', 소녀들의 여정 시작 종합, 관련도 점수: 4.383279323577881
기사 제목: 방시혁 "르세라핌뉴진스 잇는다"'알 유 넥스트' 기대되는 이유, 관련도 점수: 3.9527134895324707
기사 제목: R U Next? BTS뉴진스, 그 다음은 누구?종합, 관련도 점수: 4.369429111480713
기사 제목: 르세라핌뉴진스 이을까..'알유넥스트', 소시카라도 응원한 하이브 걸그룹종합, 관련도 점수: 4.416180610656738
기사 제목: '알 유 넥스트?' 르세라핌뉴진스 이을 하이브  걸그룹 첫 발종합 SE현장, 관련도 점수: 4.019896984100342
기사 제목: 르세라핌뉴진스 다음은?알유넥스트 서바이벌 표방 청춘 성장물종합, 관련도 점수: 4.978076934814453
기사 제목: 포토뉴진스 민지, 옆모습도 예술, 관련도 점수: 3.527411937713623
기사 제목: 미션 임파서블7 헤일리 앳웰, 서